In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
from modflowapi import ModflowApi, Callbacks, run_simulation
import numpy as np
import os
import pandas as pd
import pathlib as pl
import platform

In [ ]:
env_path = pl.Path(os.environ.get("CONDA_PREFIX", None))
assert env_path is not None, "Notebook must be run from the mf6adj Conda environment"

bin_path = "bin"
if "linux" in platform.platform().lower():
    lib_ext = ".so"
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
    lib_ext = ".dylib"
else:
    bin_path = "Scripts"
    lib_ext = ".dll"
    exe_ext = ".exe"
lib_name = env_path / f"{bin_path}/libmf6{lib_ext}"

In [ ]:
sample_frequency = "annual"  # monthly or annual
name = "sv"

In [ ]:
# load the simple model setup
adv_ws = pl.Path(f"../models/synthetic-valley-advanced-{sample_frequency}")
base_ws = pl.Path(f"../models/synthetic-valley-base-{sample_frequency}")
ws0 = adv_ws.parent / f"synthetic-valley-api-advanced-{sample_frequency}"
ws1 = base_ws.parent / f"synthetic-valley-api-base-{sample_frequency}"

## Monitor convergence

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=adv_ws, write_headers=False)

sim.memory_print_option = "all"
sim.set_sim_path(ws0)
sim.write_simulation()

In [ ]:
mf6 = ModflowApi(lib_name, working_directory=ws0)
mf6.initialize()

# maximum outer iterations
mxit_tag = mf6.get_var_address("MXITER", "SLN_1")
max_iter = mf6.get_value(mxit_tag)
max_change_tag = mf6.get_var_address("HNCG", "SLN_1")
max_change = mf6.get_value_ptr(max_change_tag)
print(max_change)

In [ ]:
xvals = np.arange(max_iter)

In [ ]:
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()

In [ ]:
idx = 0
while current_time < end_time:
    # get dt and prepare for non-linear iterations
    dt = mf6.get_time_step()
    mf6.prepare_time_step(dt)

    # convergence loop
    kiter = 0
    mf6.prepare_solve()

    print(f"timestep {idx + 1}")

    while kiter < max_iter:
        has_converged = mf6.solve()

        print(f"  {max_change[kiter]:10.4e}", end="")

        kiter += 1

        # msg = f"\tOuter iteration {kiter}: {max_change[kiter]:.4g}"
        # print(msg)
        # print(maxvals)

        if has_converged:
            msg = f"\n  Component {1}" + f" converged in {kiter}" + " outer iterations"
            print(msg)
            break

    if not has_converged:
        print("model did not converge")

    # finalize time step
    mf6.finalize_solve()

    # finalize time step and update time
    mf6.finalize_time_step()
    current_time = mf6.get_current_time()

    # increment counter
    idx += 1

In [ ]:
mf6.finalize()

## Increase precipitation

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=base_ws, write_headers=False)

sim.set_sim_path(ws1)
sim.write_simulation()

In [ ]:
def callback_function(sim, callback_step):
    """
    A demonstration function that dynamically adjusts recharge in a
    MODFLOW 6 model through the MODFLOW-API

    Parameters
    ----------
    sim : modflowapi.ApiSimulation
        A simulation object for the solution group that is
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.sv
    if callback_step == Callbacks.initialize:
        print(sim.models)

    if callback_step == Callbacks.stress_period_start:
        rcha = ml.rch_0
        spd = rcha.stress_period_data
        print(f"updating recharge: stress_period={ml.kper + 1}")
        spd["recharge"] *= 1.1

    if callback_step == Callbacks.timestep_start:
        pass

    if callback_step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass

In [ ]:
run_simulation(lib_name, ws1, callback_function, verbose=False)

### Turn on predition well when SFR flow is less than 10 cfs

In [ ]:
min_flow = 12.0

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=ws0, write_headers=False)
start_date = pd.to_datetime("1962-01-01 00:00:00")

In [ ]:
def callback_function(sim, callback_step):
    """
    A demonstration function that dynamically adjusts recharge in a
    MODFLOW 6 model through the MODFLOW-API

    Parameters
    ----------
    sim : modflowapi.ApiSimulation
        A simulation object for the solution group that is
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.sv
    if callback_step == Callbacks.initialize:
        print(sim.models)

    if callback_step == Callbacks.stress_period_start:
        if sample_frequency == "monthly":
            aug_start = 120
        else:
            aug_start = 10

        if sim.kper > aug_start:
            tag = mf6.get_var_address("DSFLOW", "SV", "SFR-1")
            arr = mf6.get_value(tag)
            arr /= -86400.0
            tag = mf6.get_var_address("INFLOW", "SV", "SFR-1")
            inflow = mf6.get_value_ptr(tag)
            if arr[-1] < min_flow:
                rate = 3.00000000e05
            else:
                rate = 0.0
            ml.prediction.stress_period_data["q"] = -rate
            inflow[0] = rate

    if callback_step == Callbacks.timestep_start:
        pass

    if callback_step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass

In [ ]:
run_simulation(lib_name, ws0, callback_function, verbose=False)

In [ ]:
gwf = sim.get_model("SV")
df = gwf.sfr.output.obs().get_dataframe(start_datetime=start_date)
df["RIV-FLOW"] /= -86400
df["RIV-SWGW"] /= -86400
df["TOTAL"] = df["RIV-SWGW"]

Q0 = df["TOTAL"].iloc[0]
df["PCT_DIFF"] = -100.0 * (df["TOTAL"] - Q0) / Q0
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Southern Boundary - Gage 1")

    ax = axs[0]
    ax.set_ylim(-5, 25)
    df["RIV-FLOW"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.axhline(min_flow, lw=0.5, ls=":", color="black")
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(-100, 100)
    df["PCT_DIFF"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")